<a href="https://colab.research.google.com/github/AccelAI/Jax-Intro-Tutorial/blob/main/JaxIntroDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The first thing we need to do is make sure that we have all the packages that we need. If you are installing on your own machine, make sure you are running python 3 and then you can install anaconda for everything we need and more or each of the following individually. 

In [ ]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install jax
!{sys.executable} -m pip install jaxlib
!{sys.executable} -m pip install tensorflow

Next lets import our first jax modules and do some basic math

In [ ]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random


In [ ]:
# Prevent GPU/TPU warning.
import jax; jax.config.update('jax_platform_name', 'cpu')